# Exercício guiado de Machine Learning

## Advanced analytics no mercado de vinho
_________

Suponha que você é um cientista de dados que trabalha na área de *advanced analytics* de uma empresa especializada na distribuição e vendas de vinhos tintos. Naturalmente, a empresa está interessada em vender vinhos que sejam percebidos como bons por grande parte do público. Pensando nisso, foi feita uma pesquisa, na qual vinhos tintos com diferentes características físico-químicas foram oferecidos a alguns voluntários, que, após experimentá-los, deram notas de 0 a 10. A base coletada contém as seguintes informações:

- Medidas de 11 variáveis físico-químicas que caracterizam cada amostra (as features do problema):
<br><br>
    - 1 - fixed acidity - medida da acidez devido à presença de ácidos orgânicos de baixa volatilidade (ácido málico, lático, tartárico ou cítrico) no vinho;
    - 2 - volatile acidity - medida da acidez devido a ácidos de baixo peso molecular (sobretudo ácido acético) presentes no vinho, que são responsáveis pelo aroma e gosto de vinagre;
    - 3 - citric acid - medida de ácido cítrico no vinho;
    - 4 - residual sugar - medida de açúcar residual presente no vinho, com origem nos resíduos de açúcar da uva que permanecem no vinho após o fim da fermentação;
    - 5 - chlorides - medida de cloretos (íons de cloro) no vinho;
    - 6 - free sulfur dioxide - medida de dióxido de enxofre livre (isto é, que não está ligado a outras moléculas) no vinho;
    - 7 - total sulfur dioxide - medida de dióxido de enxofre total (livre + porção ligada a outras moléculas) no vinho;
    - 8 - density - medida da densidade do vinho;
    - 9 - pH - medida do pH do vinho;
    - 10 - sulphates - medida de sulfatos (íons SO₄²⁻) no vinho;
    - 11 - alcohol - medida da graduação alcoólica do vinho.
<br><br>
- Além disso, há a variável resposta que no caso é um score numérico:
<br><br>
    - 12 - quality - score numérico de qualidade (de 0 a 10), produzido com base em dados sensoriais.

Com base nestes dados coletados, seu objetivo é produzir um modelo capaz de distinguir vinhos bons de ruins, com base nas medidas de suas características físico-químicas. 

Uma vez que tenhamos este modelo, caso produtoras de vinho ofereçam um novo vinho para ser vendido por sua empresa, será possível decidir de maneira mais direcionada se vale a pena passar a vender este produto ou não, de acordo com a predição de sua qualidade dada pelo modelo.

Dentro deste contexto, seu objetivo como cientista de dados é claro:

> Agregar valor ao negócio, explorando os dados que você tem à disposição.

Na primeira sprint do projeto, você e outros colegas do time de ciência de dados chegaram na seguinte _TO-DO list_ para o desenvolvimento do projeto:

- [ ] Ingestão dos dados e detalhada análise exploratória
- [ ] Formulação do problema
- [ ] Primeiro modelo baseline
- [ ] Iterações pelo ciclo de modelagem
- [ ] Compilação dos resultados para o negócio
- [ ] Comunicação dos resultados

Com base na TO-DO list acima, o time de data science quebrou a análise exploratória em algumas perguntas importantes a serem respondidas, antes da etapa de modelagem.

Agora é com você! Bom trabalho, e divirta-se! :D

_________

*Obs.: Naturalmente, o enunciado acima foi apenas uma historinha que criei pra motivar o problema em um contexto de negócio, rs. Para maiores informações sobre a coleta e origem real dos dados, veja a página do dataset no repositório UCI machine learning repository, [disponível aqui!](https://archive.ics.uci.edu/ml/datasets/wine+quality)* 

_________

Vamos começar pelos primeiros pontos da TO-DO list:

- [ ] Ingestão dos dados e detalhada análise exploratória
- [ ] Formulação do problema

_______

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

1) Leia o arquivo csv `winequality-red.csv`, construindo um Data Frame do pandas. Responda:

- Quantas linhas há no dataset?
- Quantas colunas há no dataset?
- Quais os tipos de dados em cada coluna?
- Há dados nulos (null, missing) na base?

In [ ]:
df = pd.read_csv("winequality-red.csv")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

_______

### Observação importante!

A primeira EDA pode ser feita com toda a amostra de dados que temos disponível, sem problemas.

No entanto, a partir do momento em que chegamos à conclusão de que de fato é necessário construir um modelo, é importante que façamos o **train-test split**, e:

>**Qualquer análise exploratória que, de qualquer maneira, guie o processo de construção do modelo, deve ser feita únicamente com os dados de treino!!**

Isso é importante porque, lembre-se, a base de teste tem o único propósito de nos auxiliar a estimar a performance de generalização de nosso modelo, e não deve ser usada em hipótese alguma no passo 1 (construção do modelo), pois se isso acontecer, estaremos cometendo data leakage, e a estimativa de generalização pode se invalidar!

_______

2) Utilizando a base de vinhos tintos, estude a distribuição das variáveis numéricas, calculando, para cada coluna, as principais estatísticas descritivas de posição (média, mediana, quartis, etc.) e de dispersão (std, IQR, etc.). Se desejar, visualize as distribuições de cada variável na amostra.

In [ ]:
df.describe()

In [ ]:
for col in df:

    sns.histplot(data=df, x=col, kde=True).set_title(f"Distribuição da variável {col}")
    plt.show()

_______

3) Utilizando a base de vinhos tintos, responda: existe alguma coluna com outliers? Indique qual método de detecção de outliers você utilizou, justificando seu uso.

In [ ]:
col = "alcohol"

mu, std = df[col].mean(), df[col].std()

# será outlier se |z| > 3
aux_outliers = df[col].apply(lambda x: (x - mu)/std).apply(lambda x: np.abs(x) > 3)

# outra opção, com sintaxe um pouco maior, mas talvez mais clara
# df[col].apply(lambda x: (x - mu)/std).apply(lambda x: True if np.abs(x) > 3 else False)

In [ ]:
aux_outliers.value_counts()

In [ ]:
# isto retorna apenas as linhas em que eu tenho valor "True", isso é, os outliers

aux_outliers[aux_outliers]

In [ ]:
indices_outliers = aux_outliers[aux_outliers].index.tolist()

indices_outliers

In [ ]:
df.loc[indices_outliers]

In [ ]:
# colocando tudo numa única célula

# dropando o target, pq só quero analisar as features
for col in df.drop(columns="quality"):
    
    mu, std = df[col].mean(), df[col].std()

    # será outlier se |z| > 3
    aux_outliers = df[col].apply(lambda x: (x - mu)/std).apply(lambda x: np.abs(x) > 3)
    
    indices_outliers = aux_outliers[aux_outliers].index.tolist()

    if len(indices_outliers) >= 1:
        
        print(f"A coluna {col} tem {len(indices_outliers)} outliers!")
        print("\nOs índices deles são:\n")
        print(indices_outliers)
        
    else:
        
        print(f"A coluna {col} não tem outliers!")
        
    print()
    print("="*80)
    print()

_______

4) Utilizando a base de vinhos tintos, estude os dados na coluna `quality`, que é a variável resposta do problema. Em particular, responda:

- Essa é uma variável contínua ou discreta?
- Como as notas estão distribuídas? Quais as notas mais/menos comuns?
- Faz sentido discretizar esta variável em dois níveis categóricos? 
    - Se sim, qual seria o valor de corte, e, com este corte, qual é o significado de cada nível categórico?
    - Como estes dois níveis categóricos estão distribuídos?

In [ ]:
df["quality"]

In [ ]:
sns.histplot(data=df, x="quality", kde=True);

In [ ]:
df["quality"].value_counts()

> Conclusão depois da conversa com negócio 15/08: de fato, eles esperam como resposta uma decisão binária. Por isso, decidimos seguir como um problema de classificação binária!

> Pergunta: como discretizar as notas? Ou seja, quais serão as duas classes?

_______

5) Utilizando a base de vinhos tintos, calcule e/ou visualize a correlação (utilizando a relação que achar mais adequada) entre as variáveis na base. 

Em particular, estude a correlação entre as features e o target `quality`, e responda se há correlações fortes.

Plote também a relação entre cada uma das features e o target (na forma de um scatterplot, por exemplo).

Com base nas análises acima, responda: é uma boa ideia modelar o problema como um problema de regressão? Se sim, que métodos de aprendizagem você utilizaria?

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(12, 6))

sns.heatmap(df.corr(), annot=True);

In [ ]:
df.corr()["quality"].sort_values()

In [ ]:
col="alcohol"

sns.jointplot(data=df, x=col, y="quality");

In [ ]:
for col in df.drop(columns="quality"):
    
    sns.jointplot(data=df, x=col, y="quality");

_______

6) Utilizando a base de vinhos tintos, calcule e/ou visualize (em um gráfico de barras, ou como preferir) o intervalo de confiança de 90% para a média de cada uma das variáveis físico-químicas, agrupadas pelos níveis categóricos da variável resposta `quality`. Que conclusões são possíveis tirar destes gráficos?

Sugestão: utilizar o seaborn para a visualização.

In [ ]:
for col in df.drop(columns="quality"):
    
    sns.barplot(data=df, x="quality", y=col, ci=90)
    plt.show()

_______

7) Utilizando a base de vinhos tintos, discretize a variável resposta `quality` em dois níveis categóricos para transformar o problema em um problema de classificação binária. Como valor de corte, utilize aquele que seja tal que os dois níveis categóricos estejam o mais igualmente distribuídos possível (isto é, um corte que minimize o desbalanceamento das classes). Sugestão: teste todos os valores de corte possíveis (não são muitos!)

Após a determinação do valor de corte que satisfaça às condições acima, responda: o que, qualitativamente, cada uma das duas classes representa? Esta discretização faz sentido? Se sim, para facilitar análises posteriores, nomeie as classes de acordo.

Dica: vamos usar esta nova variável resposta binária nas análises dos próximos exercícios, então sugiro que o dataframe com esta variável seja salvo num arquivo, para que ele possa ser simplesmente lido posteriormente.

In [ ]:
df["quality"].sort_values().unique()

In [ ]:
for corte in df["quality"].sort_values().unique():

    print(f"\nDistribuição de classes pra corte em nota = {corte}")

    aux_bin = df["quality"].apply(lambda x: "bom" if x > corte else "ruim")

    print(aux_bin.value_counts())
    print()
    print(aux_bin.value_counts(normalize=True)*100)

    sns.countplot(x=aux_bin)
    plt.show()

In [ ]:
df["quality_bin"] = df["quality"].apply(lambda x: "bom" if x > 5 else "ruim")

In [ ]:
df

In [ ]:
df_bin = df.drop(columns=["quality"])

df_bin

In [ ]:
df_bin.to_csv("winequality-red-binary.csv", index=False)

_______

> Na úlitma conversa com o negócio, ficou alinhado de que vamos seguir com a classificação.

> Agora vamos começar a pensar em modelo!

In [ ]:
X = df_bin.drop(columns="quality_bin")
y = df_bin["quality_bin"]

In [ ]:
X.shape

In [ ]:
y.value_counts(normalize=True)*100

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# aqui, eu junto as features e o target de novo só pra facilitar em alguns procedimentos

df_train = pd.concat([X_train, y_train], axis=1)

df_train

In [ ]:
# isso vai pra gaveta!
df_test = pd.concat([X_test, y_test], axis=1)

8) Considere a base de vinhos tintos com a variável `quality` discretizada em duas classes ("good" para score maior que 5; "bad" caso contrário). Vamos agora analisar a separabilidade das duas classes do problema. Para isso, faça:

- Visualize as distribuições das features, com indicação dos diferentes níveis categóricos do target;
- Visualize as projeções dos dados em cada um dos subespaços de pares de features, com indicação dos níveis categóricos do target;

Responda: com base nesta análise, o problema é linearmente separável?

In [ ]:
for col in X_train:

    sns.histplot(data=X_train, x=col, kde=True, hue=y_train).set_title(f"Distribuição da variável {col}")
    plt.show()

In [ ]:
sns.pairplot(df_train, hue="quality_bin");

_______

9) Considere a base de vinhos tintos com a variável `quality` discretizada em duas classes ("good" para score maior que 5; "bad" caso contrário). Separe o dataset em dados de treino (70%) e de teste (30%), estratifidando pelo target. Utilize `random_state=42` como seed, para fins de reprodutibilidade.

Apenas com os dados de treino, calcule as componentes principais do espaço de features, e responda:

- Quantas componentes principais são necessárias para que pelo menos 90% da variância do dataset seja explicada?
- Faça um scatterplot das duas primeiras componentes principais, com indicação dos níveis categóricos do target;
    - No sub-espaço das duas primeiras componentes principais, há separabilidade linear dos dados?

Dica: utilize as ferramentas do scikit-learn.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
pipe_pca = Pipeline([("ss", StandardScaler()), 
                     ("pca", PCA())])

pipe_pca.fit(X_train)

In [ ]:
pipe_pca["pca"]

In [ ]:
pipe_pca["pca"].components_

In [ ]:
pipe_pca["pca"].explained_variance_ratio_

In [ ]:
pipe_pca["pca"].explained_variance_ratio_.cumsum()

In [ ]:
X_train

In [ ]:
X_train_pca = pd.DataFrame(pipe_pca.transform(X_train), 
                           columns=[f"PC{i+1}" for i in range(X_train.shape[1])], 
                           index=X_train.index)

X_train_pca

In [ ]:
X_train_pca["PC1 PC2".split()]

In [ ]:
sns.jointplot(data=X_train_pca, x="PC1", y="PC2", hue=y_train);

_______

10) Considere a base de vinhos tintos com a variável `quality` discretizada em duas classes ("good" para score maior que 5; "bad" caso contrário). Separe o dataset em dados de treino (70%) e de teste (30%), estratifidando pelo target. Utilize `random_state=42` como seed, para fins de reprodutibilidade. Usando os dados de treino, faça:

- Agrupe os dados pelos níveis categóricos do target, e calcule a média de cada uma das features;

- Faça um teste de hipótese para determinar se, a um nível de significância de 5%, há diferença na média de cada uma das sub-amostras de cada classe, para todas as variáveis;

- Compare a distribuição das features analisando o boxplot de cada uma, separados pelas duas classes do target.

Dica: utilize as ferramentas do scipy e do scikit-learn.

In [ ]:
df_train.groupby("quality_bin").mean()

In [ ]:
df_train.groupby("quality_bin").var()

Pro teste de hipótese, vamos usar a função [ttest_ind](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html#scipy.stats.ttest_ind) do scipy.stats!

Faremos um teste t de Welch (não assumiremos variância populacional igual).

O teste que faremos será:

> $H_0: \ \mu_1 = \mu_2$

> $H_1: \ \mu_1 \neq \mu_2$

Que pode ser reescrito como:

> $H_0: \ \mu_1 - \mu_2 = 0$

> $H_1: \ \mu_1 - \mu_2 \neq 0$

________

Entendendo o que é feito no bloco de código abaixo:

In [ ]:
vinhos_bons = df_train.query("quality_bin == 'bom'")

vinhos_ruins = df_train.query("quality_bin == 'ruim'")

# o que fizemos acima com o query é equivalente a isso:
# vinhos_bons = df_train[df_train["quality_bin"] == "bom"]
# vinhos_ruins = df_train[df_train["quality_bin"] == "ruim"]

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
col = "fixed acidity"

t, p = ttest_ind(vinhos_bons[col].values, vinhos_ruins[col].values, alternative='two-sided')

print(t, p)

In [ ]:
col = "fixed acidity"

t, p = ttest_ind(vinhos_bons[col].values, vinhos_ruins[col].values, alternative='two-sided', equal_var=False)

print(t, p)

In [ ]:
sig = 0.05

if p > sig:
    print("Falha em rejeitar H0: não posso dizer que as médias são diferentes. Ou seja, não há indicios de separabilidade")
else:
    print("Rejeita H0: as médias são diferentes!!! Ou seja, indício de separabilidade das classes!")

____________

Bloco de código pro teste de hipótese:

In [ ]:
class bcolors:
    OKGREEN = '\033[92m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [ ]:
sns.kdeplot(data=df_train, x=col, hue="quality_bin")

In [ ]:
from scipy.stats import ttest_ind

# alpha de 5%
significancia = 0.05
    
# subpops de cada classe
bad_subpop = df_train.query("quality_bin == 'ruim'")
good_subpop = df_train.query("quality_bin == 'bom'")

for col in df_train.drop(columns="quality_bin"):
    
    print(f"Para a distribuição da feature {col}, temos:\n")
    
    t, p_value = ttest_ind(good_subpop[col].values, bad_subpop[col].values, alternative="two-sided", equal_var=False)
    
    print(f"t-statistic: {t:.2f}; p-value: {p_value:.2e}\n")
    
    if p_value > significancia:
        
        str_fail = f"{bcolors.FAIL}{bcolors.BOLD}"
        str_fail += "Falha em rejeitar H_0: "
        str_fail += f"parece que não há diferença na média de '{col}' em cada uma das sub-amostras 'good' e 'bad'!!"
        str_fail += f"{bcolors.ENDC}"
        
        print(str_fail)
        
    else:
        
        str_rej = f"{bcolors.OKGREEN}{bcolors.BOLD}"
        str_rej += "Rejeição da H_0: "
        str_rej += f"há diferença na média de '{col}' em cada uma das sub-amostras 'good' e 'bad'!"
        str_rej += f"{bcolors.ENDC}"
    
        print(str_rej)
        
    # =======================================
    
    sns.kdeplot(data=df_train, x=col, hue="quality_bin")
    
    # calculando as médias amostrais de cada subpop
    mu_good, mu_bad = good_subpop[col].mean(), bad_subpop[col].mean()
        
    # "C0" é o azul padrão de primeira cor; "C1" é o laranja padrão de segunda cor
    plt.axvline(x=mu_bad, color="C0", label=r"$\bar{\mu}_{ruim}=$"+f"{mu_bad:.2f}", ls=":")
    plt.axvline(x=mu_good, color="C1", label=r"$\bar{\mu}_{bom}=$"+f"{mu_good:.2f}", ls=":")

    plt.legend()
    plt.show()
    
    # =======================================
    
    print()
    print("="*80)
    print()

In [ ]:
str_fail

In [ ]:
str_rej

In [ ]:
for col in df_train.drop(columns="quality_bin"):
    
    sns.boxplot(data=df_train, x=col, y="quality_bin")
    plt.show()

Conclusão: temos um problema de classificação cujas classes não são trivialmente separadas!

Reunião com negócio, alinhamos dois pontos:

- 1°: expectativas quanto à performance do modelo, dada a dificuldade do problema;

- 2°: deixamos aberta a porta e criamos a "curiosidade" por parte do negócio de saber sobre a performance do modelo.

Avisamos pro negócio. Expectativas alinhadas.

__________
__________
__________


Uma vez que você tenha respondido às questões anteriores, você completou, talvez sem perceber, o importantíssimo (e longo!) processo de análise exploratória dos dados (EDA, do termo inglês _exploratory data analysis_)!

De fato, a etapa de EDA é importantíssima em todo projeto de ciência de dados, pois é apenas explorando os dados que de fato nos familiarizamos com o contexto do problema com o qual estamos trabalhando, o que é fundamental para o sucesso das próximas etapas, que pode envolver a criação e avaliação de modelos de machine learning, que é exatamente o que faremos agora, endereçando os próximos pontos da TO-DO list:

- [ ] Primeiro modelo baseline
- [ ] Iterações pelo ciclo de modelagem

Vamos lá!

___________

### Passo 1 - Construção do modelo

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
# modelo baseline, usamos os hiperparametros com valores default!
pipe_logit = Pipeline([("scaler", StandardScaler()),
                       ("logit", LogisticRegression())])

pipe_logit.fit(X_train, y_train)

### Passo 2 - Avaliação do modelo

# Avaliação da generalização do modelo!

### Avaliar onde estamos no tradeoff viés-variância (sobretudo no caso de alta variância (overfit))

<img src="https://estatsite.com.br/wp-content/uploads/2020/07/bias-variance-tradeoff.jpg">

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

In [ ]:
def clf_metrics(modelo, X, y_true, label, plot_conf_mat=True):
    
    print(f"\nMétricas de avaliação de {label}:\n")
    
    y_pred = modelo.predict(X)

    if plot_conf_mat:
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))

        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, ax=ax[0]) 
        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, normalize="all", ax=ax[1])
        plt.show()

    print(classification_report(y_true, y_pred))

Conforme falamos em aula, é importante que calculemos as métricas de avaliação tanto na base de treino quanto na base de teste, pra aferir se o modelo está overfitado (isto é, aferir a variância):

In [ ]:
clf_metrics(pipe_logit, X_train, y_train, "treino", plot_conf_mat=False)
print("#"*80)
clf_metrics(pipe_logit, X_test, y_test, "teste", plot_conf_mat=False)

Este modelo, no caso, sofre de underfitting (o que faz sentido, dado que a regressão logística (um modelo linear) é demasiadamente simples para os nossos dados!).

Como baseline, funciona muito bem. 

Nosso objetivo agora vai ser melhorar esta performance, aumentando um pouco a complexidadce da hipótese!

## Agora, vamos entrar no ciclo!

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# ================================
# Passo 1

pipe_rf = Pipeline([("scaler", StandardScaler()),
                    ("dt", RandomForestClassifier())])

pipe_rf.fit(X_train, y_train)

# ================================
# Passo 2

clf_metrics(pipe_rf, X_train, y_train, "treino", plot_conf_mat=False)
print("#"*80)
clf_metrics(pipe_rf, X_test, y_test, "teste", plot_conf_mat=False)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# ================================
# Passo 1

pipe_dt = Pipeline([("scaler", StandardScaler()),
                    ("dt", DecisionTreeClassifier(random_state=42))])

pipe_dt.fit(X_train, y_train)

# ================================
# Passo 2

clf_metrics(pipe_dt, X_train, y_train, "treino", plot_conf_mat=False)
print("#"*80)
clf_metrics(pipe_dt, X_test, y_test, "teste", plot_conf_mat=False)

## CLARAMENTE TEMOS MODELOS OVERFITADOS!

Por isso, **SEMPRE COMPARE AS MÉTRICAS DE TREINO COM O TESTE!**

Identificar o overfitting é relativamente fácil:

- Se tivermos um modelo perfeito na base de treino (erro 0, ou métricas de clf 1) - claramente overfitado, aprendeu até as particularidades da base de treino, deixou de conseguir generalizar!

- Se não estiver perfeito no treino, ainda assim, avalie o **gap** (isto é, a **diferença** entre as métricas de treino e teste) -- se o gap for muito alto, é pq o modelo sofre de algum grau de overfitting.

O ideal é que tenhamos um modelo que tenha um **pequeno gap** entre as métricas de treino e teste, e, que ambas sejam boas (com o trenho ligeiramente melhor que o teste). Isso é o que chamamos de "sweet spot de generalização".

Agora vamos continuar iterando no ciclo de modelagem, variando os algoritmos de aprendizagem e seus hiperparâmetros (sobretudo para controlar o balanço entre complexidade e simplicidade dos modelos!)

## Entrando no ciclo de modelagem!

In [ ]:
y_pred = pipe_dt.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
classification_report(y_test, y_pred, output_dict=False)

In [ ]:
dict_metricas = classification_report(y_test, y_pred, output_dict=True)

dict_metricas["weighted avg"]["f1-score"]

A mesma função de antes, mas agora com o dicionário do classificarion report retornada, pra gente conseguir extrair as métricas!

In [ ]:
def clf_metrics_com_return(modelo, X, y_true, label, plot_conf_mat=True, print_cr=True):
    
    if print_cr:
        print(f"\nMétricas de avaliação de {label}:\n")
    
    y_pred = modelo.predict(X)

    if plot_conf_mat:
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))

        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, ax=ax[0]) 
        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, normalize="all", ax=ax[1])
        plt.show()

    if print_cr:
        print(classification_report(y_true, y_pred))
    
    return classification_report(y_true, y_pred, output_dict=True)

In [ ]:
from sklearn.svm import SVC

In [ ]:
pipe_logit = Pipeline([("scaler", StandardScaler()),
                       ("logit", LogisticRegression(random_state=42))])

pipe_rf = Pipeline([("scaler", StandardScaler()),
                    ("rf", RandomForestClassifier(random_state=42))])

pipe_dt = Pipeline([("scaler", StandardScaler()),
                    ("dt", DecisionTreeClassifier(random_state=42))])

pipe_svm = Pipeline([("scaler", StandardScaler()),
                     ("svm", SVC(random_state=42))])

# =======================================

dict_pipes = {"logit" : pipe_logit,
              "random forest" : pipe_rf,
              "decision tree" : pipe_dt,
              "svm" : pipe_svm}

# =======================================
# experimento!

resultado_experimentos = {"estimador" : [],
                          "f1_treino" : [],
                          "f1_teste" : []}

print_progress = False

for label, pipe in dict_pipes.items():
    
    if print_progress:
        print("\n")
        print("="*80)
        print(f"Estimador: {label}".center(80))
        print("(com hiperparâmetros default)".center(80))
        print("="*80)
        print("\n")

    # ================================
    
    pipe.fit(X_train, y_train)

    # ================================
    # Passo 2

    dict_metricas_treino = clf_metrics_com_return(pipe, X_train, y_train, "treino", plot_conf_mat=False, print_cr=False)
    
    if print_progress:
        print("#"*80)
        
    dict_metricas_teste = clf_metrics_com_return(pipe, X_test, y_test, "teste", plot_conf_mat=False, print_cr=False)
    
    # pegar as métricas pra salvar abaixo!
    f1_treino = dict_metricas_treino["weighted avg"]["f1-score"]
    f1_teste = dict_metricas_teste["weighted avg"]["f1-score"]
    
    # ================================
    # guardando os resultados do experimento
    
    resultado_experimentos["estimador"].append(label)
    resultado_experimentos["f1_treino"].append(f1_treino)
    resultado_experimentos["f1_teste"].append(f1_teste)
    
    
df_results = pd.DataFrame(resultado_experimentos)

df_results["gap"] = (df_results["f1_treino"] - df_results["f1_teste"]).apply(lambda x: x if x > 0 else np.inf)

df_results = df_results.sort_values("f1_teste", ascending=False).sort_values("gap")

df_results

In [ ]:
def experimentos_ciclo_de_modelagem(dict_pipes, 
                                    plot_conf_mat=False, print_cr=False,
                                    print_progress = False):
    
    resultado_experimentos = {"estimador" : [],
                              "f1_treino" : [],
                              "f1_teste" : []}

    for label, pipe in dict_pipes.items():

        if print_progress:
            print("\n")
            print("="*80)
            print(f"Estimador: {label}".center(80))
            print("(com hiperparâmetros default)".center(80))
            print("="*80)
            print("\n")

        # ================================

        pipe.fit(X_train, y_train)

        # ================================
        # Passo 2

        dict_metricas_treino = clf_metrics_com_return(pipe, X_train, y_train, "treino", 
                                                      plot_conf_mat=plot_conf_mat, print_cr=print_cr)

        if print_progress:
            print("#"*80)

        dict_metricas_teste = clf_metrics_com_return(pipe, X_test, y_test, "teste", 
                                                     plot_conf_mat=plot_conf_mat, print_cr=print_cr)

        # pegar as métricas pra salvar abaixo!
        f1_treino = dict_metricas_treino["weighted avg"]["f1-score"]
        f1_teste = dict_metricas_teste["weighted avg"]["f1-score"]

        # ================================
        # guardando os resultados do experimento

        resultado_experimentos["estimador"].append(label)
        resultado_experimentos["f1_treino"].append(f1_treino)
        resultado_experimentos["f1_teste"].append(f1_teste)


    df_results = pd.DataFrame(resultado_experimentos)

    df_results["gap"] = (df_results["f1_treino"] - df_results["f1_teste"]).apply(lambda x: x if x > 0 else np.inf)

    df_results = df_results.sort_values("f1_teste", ascending=False).sort_values("gap")

    return df_results

___________

Agora sim, tudo mais condensado:

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
pipe_logit = Pipeline([("scaler", StandardScaler()),
                       ("logit", LogisticRegression(random_state=42))])

pipe_rf = Pipeline([("scaler", StandardScaler()),
                    ("rf", RandomForestClassifier(random_state=42))])

pipe_dt = Pipeline([("scaler", StandardScaler()),
                    ("dt", DecisionTreeClassifier(random_state=42))])

pipe_svm = Pipeline([("scaler", StandardScaler()),
                     ("svm", SVC(random_state=42))])

pipe_xbgoost = Pipeline([("scaler", StandardScaler()),
                         ("xgboost", XGBClassifier(random_state=42))])

pipe_lgbm = Pipeline([("scaler", StandardScaler()),
                      ("lgbm", LGBMClassifier(random_state=42))])

pipe_knn = Pipeline([("scaler", StandardScaler()),
                     ("knn", KNeighborsClassifier())])

# =======================================

dict_pipes = {"logit" : pipe_logit,
              "random_forest" : pipe_rf,
              "decision_tree" : pipe_dt,
              "svm" : pipe_svm,
              "xgboost" : pipe_xbgoost,
              "lgbm" : pipe_lgbm,
              "knn" : pipe_knn}

# =======================================

df_results = experimentos_ciclo_de_modelagem(dict_pipes, 
                                             plot_conf_mat=False, print_cr=False,
                                             print_progress = False)

df_results

____________

### Otimização de hiperparâmetros

- Começo com um random search, pra identificar **regiões promissoras no espçao de hiperparâmetros**;

- Depois de encontrar estas regiões promissoros, uso o grid search pra fazer um **ajuste fino** nas redondezas da região promissora.

Tudo isso, se for relativamente rápido de passar pelos processos acima. 

Se demorar muito (é o caso, por exemplo, de bases muito grandes), parto direto pra otimização baeysiana.

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold

In [ ]:
np.arange(100, 1501, 1)

In [ ]:
np.arange(2, 9, 1)

In [ ]:
# pipeline
pipe_rf = Pipeline([("scaler", StandardScaler()),
                    ("rf", RandomForestClassifier(n_jobs=-1, random_state=42))])

# espaço de hiperparâmetros 
# (esta definição vem do conhecimento que temos do método e de seus hiperparâmetros)
params_distributions = {"rf__n_estimators" : np.arange(100, 1501, 1), 
                        "rf__max_depth" : np.arange(2, 9, 1)}

# estratégia de cross validation 
splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# montamos o objeto do random search
rand_grid_rf = RandomizedSearchCV(pipe_rf, 
                                  params_distributions, 
                                  n_iter=20,
                                  cv=splitter,
                                  scoring="f1_weighted",
                                  verbose=10,
                                  n_jobs=-1,
                                  random_state=42,
                                  return_train_score=True)

rand_grid_rf.fit(X_train, y_train)

In [ ]:
pd.DataFrame(rand_grid_rf.cv_results_).sort_values("rank_test_score")

In [ ]:
rand_grid_rf.best_params_

In [ ]:
def calc_best_params_delta(grid, peso_delta=0.5, print_deltas=False):
    '''
    to-do: docsting
    
    - grid: é um objeto gridsearch já fitado!
    '''
    
    cv_results_df = pd.DataFrame(grid.cv_results_)

    aux = cv_results_df[['mean_train_score', 'mean_test_score']].copy()

    # esse será nosso novo critério: comparando treino e teste!!!
    # assim, evitamos overfitting!
    aux["delta"] = (aux["mean_train_score"] - aux["mean_test_score"]).abs()

    # vamos normalizar tanto as métricas quanto o delta pro intervalo (0.1, 0.9)
    # (nao deixei (0, 1) pra nao zerar as coisas)
    # com isso, podemos tratar de maneira unificada tanto problemas de regressão quanto classificação
    aux_norm = pd.DataFrame(MinMaxScaler((0.1, 0.9)).fit_transform(aux), 
                            columns=[f"{x}_norm" for x in aux.columns], index=aux.index)

    # considere:
    # - w: o peso que colocamos no delta (1-w, portanto, é o peso que colocamos na métrica de teste);
    # - d: delta;
    # - m: a métrica de teste;
    # a métrica final, que é MAXIMIZADA, é a seguinte:
    # (w*(1-d)) + ((1-w)*m)
    # quero maximizar essa métrica final, com o objetivo de MAXIMIZAR métrica de teste e ao mesmo tempo o MINIMIZAR o delta
    # pra esse fim, somo as duas contribuições, tomando a métrica em si, e o complementar do delta, ponderados respectivamente
    # com isso, consigo um ponto de equilibrio legal!
    aux_norm["metrica_criterio_final"] = (peso_delta*(1-aux_norm["delta_norm"])) + ((1-peso_delta)*aux_norm["mean_test_score_norm"])

    aux = pd.concat([aux, aux_norm], axis=1).sort_values("metrica_criterio_final", ascending=False)
    
    if print_deltas:
        display(aux)

    # esse é o indice correspondente à melhor métrica de critério final
    # (note que já ordenamos acima, então a melhor métrica é a primeira linha!)
    num_combinacao_melhor_delta = aux.iloc[0, :].name

    # isso dá os melhores parametros, segundo o critério do delta!!
    best_params_delta = cv_results_df.loc[num_combinacao_melhor_delta, "params"]

    return best_params_delta


In [ ]:
rand_grid_rf.best_params_

In [ ]:
pipe_rf = Pipeline([("scaler", StandardScaler()),
                    ("rf", RandomForestClassifier())]).set_params(**{'rf__n_estimators': 365, 'rf__max_depth': 7})

pipe_rf

In [ ]:
pipe_rf = Pipeline([("scaler", StandardScaler()),
                    ("rf", RandomForestClassifier(random_state=42))]).set_params(**{'rf__n_estimators': 365,
                                                                                    'rf__max_depth': 7})

pipe_rf.fit(X_train, y_train)

# ================================
# Passo 2

clf_metrics(pipe_rf, X_train, y_train, "treino", plot_conf_mat=False)
print("#"*80)
clf_metrics(pipe_rf, X_test, y_test, "teste", plot_conf_mat=False)

In [ ]:
calc_best_params_delta(rand_grid_rf, peso_delta=0.5, print_deltas=False)

In [ ]:
pipe_rf = Pipeline([("scaler", StandardScaler()),
                    ("rf", RandomForestClassifier(random_state=42))]).set_params(**{'rf__n_estimators': 683, 
                                                                                    'rf__max_depth': 4})

pipe_rf.fit(X_train, y_train)

# ================================
# Passo 2

clf_metrics(pipe_rf, X_train, y_train, "treino", plot_conf_mat=False)
print("#"*80)
clf_metrics(pipe_rf, X_test, y_test, "teste", plot_conf_mat=False)

_________

Podemos pegar essa região promissora que saiu do random search acima (`{'rf__n_estimators': 683, 'rf__max_depth': 4}`), e buscar com o grid search na vizinhança dela (um ajuste fino!)

Veja que o grid search (que testa todas as combinações sistematicamente) já vai incluir os hiperparametros encontrados acima!

Ou seja, só dá pra melhorar! Piorar não vai!

In [ ]:
list(range(683-3, 683+3+1))

In [ ]:
# pipeline
pipe_rf = Pipeline([("scaler", StandardScaler()),
                    ("rf", RandomForestClassifier(n_jobs=-1, random_state=42))])

# grade de hiperparâmetros na "redondeza" da região promissora encontrada acima!
parameters_grid = {"rf__n_estimators" : range(683-3, 683+3+1), 
                   "rf__max_depth" : [3, 4, 5]}

# estratégia de cross validation 
splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# montamos o objeto do random search
grid_rf = GridSearchCV(pipe_rf, 
                        parameters_grid, 
                        cv=splitter,
                        scoring="f1_weighted",
                        verbose=10,
                        n_jobs=-1,
                        return_train_score=True)

grid_rf.fit(X_train, y_train)

In [ ]:
# olhando só pra métrica de teste
grid_rf.best_params_

In [ ]:
pipe_rf = Pipeline([("scaler", StandardScaler()),
                    ("rf", RandomForestClassifier(random_state=42))]).set_params(**{'rf__max_depth': 5, 
                                                                                    'rf__n_estimators': 680})

pipe_rf.fit(X_train, y_train)

# ================================
# Passo 2

clf_metrics(pipe_rf, X_train, y_train, "treino", plot_conf_mat=False)
print("#"*80)
clf_metrics(pipe_rf, X_test, y_test, "teste", plot_conf_mat=False)

In [ ]:
# olhando tanto pro teste quanto pro delta
calc_best_params_delta(grid_rf, peso_delta=0.5, print_deltas=False)

In [ ]:
pipe_rf = Pipeline([("scaler", StandardScaler()),
                    ("rf", RandomForestClassifier(random_state=42))]).set_params(**{'rf__max_depth': 4, 
                                                                                    'rf__n_estimators': 680})

pipe_rf.fit(X_train, y_train)

# ================================
# Passo 2

clf_metrics(pipe_rf, X_train, y_train, "treino", plot_conf_mat=False)
print("#"*80)
clf_metrics(pipe_rf, X_test, y_test, "teste", plot_conf_mat=False)

## Auto ML

In [ ]:
# baseline e ciclo de modelagem!

In [ ]:
# incluir PCA na pipeline

In [ ]:
# detalhar o gridsearch, montagem do espaço de hiperparametros

In [ ]:
# tradeoff precision-recall e cutoff

__________
__________
__________


Agora que já passamos um bom tempo no ciclo de modelagem, e temos ótimos resultados, precisamos reportá-los para o negócio. Isto é, falta endereçar os dois últimos pontos da TO-DO list:

- [ ] Compilação dos resultados para o negócio
- [ ] Comunicação dos resultados

Para isso, use e abuse de ferramentas de dataviz, faça uma apresentação no PPT, enfim, o que for necessário para passar a mensagem para o negócio de maneira efetiva e precisa. E, lembre-se, a equipe de negócio não é técnica, então trate de usar uma linguagem acessível e com o mínimo de tecnicalidades --- mas esteja preparado para perguntas técnicas (talvez com alguns slides ocultos no final da apresentação), pois nunca sabemos quando perguntas assim podem aparecer!

In [ ]:
# comunicação de resultados

__________
__________
__________
